In [54]:
# Initial Imports
import os
from dotenv import load_dotenv

import pandas as pd
from datetime import datetime

import finnhub
import ta #Technical Analysis library

load_dotenv()

True

In [8]:
# Setting up Finnhub API
finnhub_api_key = os.getenv("FINNHUB_API_KEY")
finnhub_sandbox_key = os.getenv("FINNHUB_SANDBOX_KEY")
finnhub_client = finnhub.Client(api_key = finnhub_sandbox_key)

In [24]:
def getUNIX(date):
    """
    Input date in YYYY-MM-DD format (as a string) and returns the associated UNIX timestamp
    """
    # Parsing the input date
    dateparts = date.split("-")
    year = int(dateparts[0])
    month = int(dateparts[1])
    day = int(dateparts[2])
    
    unix = int((datetime(year, month, day) - datetime(1970,1,1)).total_seconds())
    return unix

In [27]:
def getYMD(unix):
    """
    Input a UNIX timestamp and returns a date in the format of YYYYMMDD
    Any additional hours, minutes, or seconds are dropped
    """
    ts = int(unix)
    return datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')

In [43]:
def getOHLCV(ticker, startDate, endDate):
    """
    Input a ticker, startDate (YYYY-MM-DD), endDate (YYYY-MM-DD)
    Returns daily open, high, low, close, and volume (in that order) in a pandas dataframe for the given criteria
    """
    # @TODO: Error handling -- check to see that endDate is after startDate, endDate has already passed, etc.
    
    # Converting to UNIX timestamp
    startDate = getUNIX(startDate)
    endDate = getUNIX(endDate)
    
    # Calling Finnhub API for candles data
    candlesData = finnhub_client.stock_candles(ticker, 'D', startDate, endDate)
    OHLCV = pd.DataFrame(candlesData)
    
    # Dropping the column denoting status of response and any null fields
    OHLCV.drop(columns = "s", inplace = True)
    OHLCV.dropna(inplace = True)
    
    # Renaming columns for ease of interpretation
    OHLCV = OHLCV.rename(columns = {
        "c":"close",
        "h":"high",
        "l":"low",
        "o":"open",
        "t":"date",
        "v":"volume"
    })
    
    # Converting UNIX timestamp to date and setting date as index
    OHLCV["date"] = OHLCV["date"].apply(getYMD)
    OHLCV.set_index(OHLCV["date"], inplace = True)
    OHLCV.drop(columns = "date", inplace = True)
    
    # Reordering columns to match OHLCV
    OHLCV = OHLCV[["open", "high", "low", "close", "volume"]]
    
    return OHLCV

In [118]:
def getTechIndicators(OHLCV):
    """
    Input OHLCV dataframe. Please note that due to the windows of the SMAs, at least 200 day's worth of data will be required. 
    Rows with null values will be automatically dropped
    
    Calculates the following technical indicators:
    
    Simple Moving Average with 20, 50, and 100 day windows
    Moving Average Convergence Divergence (MACD)

    On Balance Volume (OBV)
    Chaikin Money Flow (CMF)
    
    Awesome Oscillator (AOsc)
    
    Relative Strength Index (RSI)
    Stochastic Oscillator (SOsc)

    """
    # @TODO: Validate OHLCV has at least 200 rows
    
    # Calculating the Simple Moving Averages
    OHLCV["SMA20"] = OHLCV["close"].rolling(window = 20).mean()
    OHLCV["SMA50"] = OHLCV["close"].rolling(window = 50).mean()
    OHLCV["SMA100"] = OHLCV["close"].rolling(window = 100).mean()
    
    # Calculating MACD
    ewm26 = OHLCV['close'].ewm(halflife = 26).mean()
    ewm12 = OHLCV['close'].ewm(halflife = 12).mean()
    OHLCV["MACD"] = ewm12 - ewm26
    
    # Calculating on balance volume
    OHLCV["OBV"] = ta.volume.on_balance_volume(OHLCV["close"], OHLCV["volume"])
    
    # Calculating Chaikin Money Flow
    OHLCV["CMF"] = ta.volume.ChaikinMoneyFlowIndicator(
        high = OHLCV["high"], 
        low = OHLCV["low"], 
        close = OHLCV["close"], 
        volume = OHLCV["volume"]).chaikin_money_flow()
    
    # Calculating Awesome Oscillator
    OHLCV["AOsc"] = ta.momentum.AwesomeOscillatorIndicator(
        high = OHLCV["high"], 
        low = OHLCV["low"]).awesome_oscillator()
    
    # Calculating RSI
    OHLCV["RSI"] = ta.momentum.RSIIndicator(close = OHLCV["close"]).rsi()
    
    # Calculating Stochastic Oscillator
    OHLCV["SOsc"] = ta.momentum.StochasticOscillator(high = , low = OHLCV["close"], close = OHLCV["close"])
    
    return OHLCV

In [119]:
OHLCV = getOHLCV("AAPL", "2019-01-20", "2020-01-20")
getTechIndicators(OHLCV)

,open,high,low,close,volume,SMA20,SMA50,SMA100,MACD,OBV,CMF,AOsc,RSI
date,,,,,,,,,,,,,
2019-01-22,175.949993,176.310001,171.719999,172.485008,547091460,NaN,NaN,NaN,0.000000,547091460,NaN,NaN,NaN
2019-01-23,173.430004,174.509995,170.639992,173.159998,416350260,NaN,NaN,NaN,0.005246,963441720,NaN,NaN,NaN
2019-01-24,173.384995,173.789995,170.729994,171.764992,457947864,NaN,NaN,NaN,-0.007760,505493856,NaN,NaN,NaN
2019-01-25,174.914995,177.884995,173.610008,177.479994,603862056,NaN,NaN,NaN,0.054423,1109355912,NaN,NaN,NaN
2019-01-28,175.275003,175.860008,172.844999,175.860008,471457044,NaN,NaN,NaN,0.069318,637898868,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-13,350.595016,356.714985,350.055004,356.579990,549390996,328.720501,309.201301,280.424251,21.680563,15486906744,0.277410,29.104545,85.433884
2020-01-14,356.264992,357.254997,351.180004,351.764992,731762208,330.831000,310.639501,281.549701,22.058440,14755144536,0.176376,30.884161,77.048333
2020-01-15,350.819996,354.959988,348.254997,350.235008,548655876,332.599500,311.888701,282.661651,22.327578,14206488660,0.132151,31.935438,74.544573
